In [18]:
import pandas as pd

In [19]:
datasets = ["hungarian", "cleveland", "switzerland", "long-beach-va"]

In [20]:
def readRawData(filePath:str):
    with open(filePath) as file:
        dataString = file.read()
        dataString = dataString.replace("\n"," ")
        dataString = dataString.replace("name ","name\n")
        dataString = dataString.replace(" ",",")
        return dataString

In [21]:
from io import StringIO
df = pd.DataFrame()
for dataset in datasets:
    dataset_df = pd.read_csv(StringIO(readRawData("./Data/"+ dataset +".data")), header=None, sep=",")
    dataset_df['dataset'] = dataset
    df = pd.concat([df,dataset_df ], ignore_index=True)

In [22]:
df.columns=["id", "ccf", "age", "sex", "painloc", "painexer" , "relrest" , "pncaden" , "cp", "trestbps", "htn", "chol", "smoke", "cigs", "years", "fbs", "dm", "famhist", "restecg", "ekgmo", "ekgday", "ekgyr", "dig", "prop", "nitr", "pro", "diuretic", "proto", "thaldur", "thaltime", "met", "thalach", "thalrest", "tpeakbps", "tpeakbpd", "dummy", "trestbpd", "exang", "xhypo", "oldpeak", "slope", "rldv5", "rldv5e", "ca", "restckm", "exerckm", "restef", "restwm", "exeref", "exerwm", "thal", "thalsev", "thalpul", "earlobe", "cmo", "cday", "cyr", "num", "lmt", "ladprox", "laddist", "diag", "cxmain", "ramus", "om1", "om2", "rcaprox", "rcadist", "lvx1", "lvx2", "lvx3", "lvx4", "lvf", "cathef", "junk", "name", "dataset"]

In [23]:
df = df.replace(-9, float('nan'))

In [24]:
df.describe()

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,...,om2,rcaprox,rcadist,lvx1,lvx2,lvx3,lvx4,lvf,cathef,junk
count,899.000000,899.0,899.000000,899.000000,617.000000,617.000000,613.000000,0.0,899.000000,840.000000,...,327.000000,654.000000,629.000000,880.000000,880.000000,880.000000,880.000000,883.000000,311.000000,119.000000
mean,957.235818,0.0,53.480534,0.790879,0.920583,0.593193,0.672104,NaN,3.253615,132.101190,...,1.067278,1.342508,1.171701,1.020455,1.032955,1.132955,1.611364,1.178935,27.623119,5.869748
std,1204.015482,0.0,9.435894,0.406908,0.270607,0.491637,0.469830,NaN,0.928499,19.151127,...,0.250887,0.474912,0.377421,0.277384,0.415902,0.703837,1.722199,0.512572,31.675295,1.650914
min,1.000000,0.0,28.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.220000,3.300000
25%,116.000000,0.0,47.000000,1.000000,1.000000,0.000000,0.000000,NaN,3.000000,120.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.685000,4.800000
50%,266.000000,0.0,54.000000,1.000000,1.000000,1.000000,1.000000,NaN,4.000000,130.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.820000,5.600000
75%,1207.500000,0.0,60.000000,1.000000,1.000000,1.000000,1.000000,NaN,4.000000,140.000000,...,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,63.000000,6.900000
max,5002.000000,0.0,77.000000,1.000000,1.000000,1.000000,1.000000,NaN,4.000000,200.000000,...,2.000000,2.000000,2.000000,7.000000,10.000000,8.000000,8.000000,5.000000,86.000000,11.300000


In [25]:
df.head()

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,...,rcadist,lvx1,lvx2,lvx3,lvx4,lvf,cathef,junk,name,dataset
0,1254.0,0.0,40.0,1.0,1.0,0.0,0.0,NaN,2.0,140.0,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian
1,1255.0,0.0,49.0,0.0,1.0,0.0,0.0,NaN,3.0,160.0,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian
2,1256.0,0.0,37.0,1.0,1.0,0.0,0.0,NaN,2.0,130.0,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian
3,1257.0,0.0,48.0,0.0,1.0,1.0,1.0,NaN,4.0,138.0,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian
4,1258.0,0.0,54.0,1.0,1.0,0.0,1.0,NaN,3.0,150.0,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian


# Explore how many NaNs are within one coloumn for each attribute.

In [26]:
(df.loc[ : , df.columns != 'dataset'].isna()).join(df['dataset']).groupby("dataset").sum()

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,...,rcaprox,rcadist,lvx1,lvx2,lvx3,lvx4,lvf,cathef,junk,name
dataset,,,,,,,,,,,,,,,,,,,,,
cleveland,0,0,0,0,282,282,282,282,0,0,...,0,0,0,0,0,0,0,282,282,0
hungarian,1,1,1,1,1,1,1,295,1,2,...,245,270,1,1,1,1,1,267,295,1
long-beach-va,1,1,1,1,1,1,5,201,1,57,...,2,2,3,3,3,3,4,24,82,1
switzerland,0,0,0,0,0,0,0,123,0,2,...,0,0,17,17,17,17,13,17,123,0


In [27]:
df.drop(["painloc", "painexer", "relrest", "pncaden", "trestbps", "htn", "chol", "smoke", "cigs", "years", "fbs",  "dm", "famhist", "ekgmo", "ekgday", "ekgyr", "dig", "prop", "nitr", "pro", "diuretic", "proto", "thaldur", "thaltime", "met", "slope", "rldv5", "rldv5e", "ca", "restckm", "exerckm", "restef", "restwm", "exeref", "exerwm", "thal", "thalsev", "thalpul", "earlobe", "lmt", "ladprox", "laddist", "diag", "cxmain", "ramus", "om1", "om2", "rcaprox", "rcadist", "lvx1", "lvx2", "lvx3", "lvx4", "lvf", "cathef", "junk","thal" ], inplace=True, axis=1)

In [28]:
df.isna().sum()

id           2
ccf          2
age          2
sex          2
cp           2
restecg      4
thalach     57
thalrest    58
tpeakbps    65
tpeakbpd    65
dummy       61
trestbpd    61
exang       57
xhypo       60
oldpeak     64
cmo         13
cday        11
cyr         11
num          2
name         2
dataset      0
dtype: int64

In [29]:
df.dropna(inplace=True, axis=0, how='any')

In [30]:
(df.loc[ : , df.columns != 'dataset'].isna()).join(df['dataset']).groupby("dataset").sum()

,id,ccf,age,sex,cp,restecg,thalach,thalrest,tpeakbps,tpeakbpd,dummy,trestbpd,exang,xhypo,oldpeak,cmo,cday,cyr,num,name
dataset,,,,,,,,,,,,,,,,,,,,
cleveland,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
hungarian,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
long-beach-va,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
switzerland,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
df.shape

(815, 21)

In [32]:
df.drop(['id', 'name'], inplace=True, axis=1)

In [33]:
df

,ccf,age,sex,cp,restecg,thalach,thalrest,tpeakbps,tpeakbpd,dummy,trestbpd,exang,xhypo,oldpeak,cmo,cday,cyr,num,dataset
0,0.0,40.0,1.0,2.0,0.0,172.0,86.0,200.0,110.0,140.0,86.0,0.0,0.0,0.0,12.0,20.0,84.0,0.0,hungarian
1,0.0,49.0,0.0,3.0,0.0,156.0,100.0,220.0,106.0,160.0,90.0,0.0,0.0,1.0,11.0,20.0,84.0,1.0,hungarian
2,0.0,37.0,1.0,2.0,1.0,98.0,58.0,180.0,100.0,130.0,80.0,0.0,0.0,0.0,11.0,26.0,84.0,0.0,hungarian
3,0.0,48.0,0.0,4.0,0.0,108.0,54.0,210.0,106.0,138.0,86.0,1.0,0.0,1.5,9.0,30.0,84.0,3.0,hungarian
4,0.0,54.0,1.0,3.0,0.0,122.0,74.0,130.0,100.0,150.0,90.0,0.0,1.0,0.0,7.0,30.0,84.0,0.0,hungarian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,0.0,62.0,1.0,4.0,1.0,138.0,86.0,202.0,98.0,158.0,90.0,1.0,0.0,0.0,1.0,26.0,87.0,1.0,long-beach-va
894,0.0,46.0,1.0,4.0,0.0,126.0,88.0,174.0,114.0,134.0,90.0,0.0,0.0,0.0,7.0,28.0,83.0,2.0,long-beach-va
895,0.0,54.0,0.0,4.0,1.0,154.0,83.0,158.0,84.0,127.0,78.0,0.0,0.0,0.0,6.0,29.0,83.0,1.0,long-beach-va
897,0.0,55.0,1.0,4.0,1.0,100.0,74.0,210.0,100.0,122.0,70.0,0.0,0.0,0.0,4.0,17.0,86.0,2.0,long-beach-va


In [34]:
# from pandas_profiling import ProfileReport
# ProfileReport(df, title='Pandas Profiling Report')

ModuleNotFoundError: No module named 'pandas_profiling'

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
df['num'] = labelEncoder.fit_transform(df['num'])

In [ ]:
X = df.loc[:,(df.columns!= 'num') & (df.columns != 'dataset')]
y = df['num']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from statistics import mean, stdev
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

stratifiedKFold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []

catboost = CatBoostClassifier()
minMaxScaler = MinMaxScaler()
X = minMaxScaler.fit_transform(X)

for train_index, test_index in stratifiedKFold.split(X, y):
    x_train_fold, x_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    catboost.fit(x_train_fold, y_train_fold)
    lst_accu_stratified.append(catboost.score(x_test_fold, y_test_fold))

# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))